In this section of the lab, we'll use some data from NASA to determine the temperature of the cosmic microwave background (CMB).

The CMB is the radiation left over from the big bang, and it exhibits a nearly perfect black body distribution. Deviation for a perfectly uniform distibution as a function of direction (there are very small temperature fluctuations as you look in different directions) are of intense interest in cosmology.

We'll be fitting the monopole/uniform background data, which we have downloaded here

Data source: https://lambda.gsfc.nasa.gov/product/cobe/firas_products.cfm

If you'd like to read more about COBE and FIRAS, here are some great starting points:

https://en.wikipedia.org/wiki/Cosmic_Background_Explorer

https://lambda.gsfc.nasa.gov/product/cobe/firas_overview.cfm

Execute the cells below to import some required libraries

In [ ]:
%matplotlib notebook
import numpy as np
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt

In class we discussed the intensity distribution of a black body radiator. To use the NASA data, we'll need to instead work with spectral irradiance per solid angle $B$.

$$ B(f, T) = \frac{2 h f^3}{c^2}\frac{1}{exp\left(\frac{hf}{k_BT}\right) -1}$$

It describes the intensity per frequency per solid angle at a given temperature. 


Solid angle wikipedia article: https://en.wikipedia.org/wiki/Solid_angle

**Question 1** 
What are the SI units of $B$?


Next, define a function to evaluate $B(f, t)$, along with some constants we need

In [ ]:
h = 6.62606e-34 # m^2 kg / s      Planck's constant
c = 2.998e8 # m / s               Speed of light
kB = 1.38065e-23 # J / k

def bb_spectrum_freq(freqs, T):
    return 2 * h * np.power(freqs, 3) / ( c * c * (np.exp(h*freqs/(kB * T)) - 1))


Next, we'll load the data. It contains frequencies $f$ (`freqs`) at which $B$ (`bs`) was measured, along with uncertainties in $B$, $\sigma$ (`sigmas`).

In [ ]:
def load_firas_data():
    freqs = list()
    bs = list()
    sigmas = list()
    Jy_conversion = 1e-20 # 1MJy to W/m^2

    with open('data/firas_monopole_spec_v1.txt', 'rt') as data_file:
        for line in data_file:
            if line.startswith('#'):
                continue
            freq, intensity, resid, sigma,  __ = map(float, line.split())
            freqs.append(freq)
            bs.append(intensity + resid * 1e-3)
            sigmas.append(sigma)
    sigmas = np.array(sigmas) * Jy_conversion * 1e-3
    freqs = np.array(freqs) *  c * 100 # convert to hertz (from cm-1)
    bs = np.array(bs) * (Jy_conversion)
    
    return freqs, bs, sigmas

freqs, bs, sigmas = load_firas_data()

Now let's plot it!

In [ ]:
plt.figure()
plt.errorbar(freqs, bs, sigmas, fmt='.')
plt.ylabel('Irradiance ($W m^{-2} Hz^{-1}str^{-1}$)')
plt.xlabel('$f (Hz)$')
plt.title('COBE FIRAS Cosmic Microwave Background')
plt.show()

Wien's displacement Law for frequencies states that

$$f_{peak}/T = 5.879 \times 10^{10} Hz / K $$


**Question 2**

What is value of f_peak (find from the graph above)? What temperature does this predict for the CMB?

In [ ]:
f_peak = # ENTER YOUR VALUE
T_peak =  f_peak / 5.879e10 # compute T

print('T = {}'.format(T_peak))

Next, we'll plot this distribution versus our data. The function `bb_spectrum_freq(frequencies, T)` takes an array of frequencies (in Hz) as its first argument and a temperature (in Kelvin). It returns the values of the black body irradiance spectrum evaluated at the array of frequencies.


We'll use a few functions below to generate our plot.

`np.linspace(min_value, max_value, number_samples)` will return an array of evenly spaced values between min_value and max_value.

`plt.plot(x, y, label='label')` will plot x vs y in the current figure.

`plt.errorbar(x, y, yerr, fmt='.')` will plot x vs y and add error bars of width `yerr` to the plot

Plot the theoretical data at the temperature you computed above against the FIRE data.

In [ ]:
plt.figure(figsize=(8, 9))
f_theory = np.linspace(5e9, 8e11, 1000)

bb_theory = bb_spectrum_freq(f_theory, T=T_peak)

plt.plot(f_theory, bb_theory, '-k', label='Theory {:0.3f}K'.format(T_peak))
plt.errorbar(freqs, bs, 6*sigmas, fmt='.r', label='COBE FIRAS')#  plot 95% confidence error bars (+/- 3 sigma)
plt.legend()

plt.ylabel('B ($W m^{-2} Hz^{-1}str^{-1}$)')
plt.xlabel('$f (Hz)$')
plt.legend()
plt.show()

**Question 3**

How well does this fit predict the irradiance distribution? How could you get a better estimate?

Answer:

Next, we'll fit this using a method called non-linear least squares. You can read about it [here](https://en.wikipedia.org/wiki/Non-linear_least_squares).

The specific implementation we'll use is from scipy ("Scientific Python"). This is an open source library that contains an incredibly powerful set of tools for scientific analysis and it is absolutely free! The scipy documentation for the `curve_fit` function we will use is [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)

The way non-linear least squares (as well a most regression methods work) is that you provide a set of data points of the dependent variable (`ydata`) at a set of independent variables (`xdata`). You also specify the uncertainty in your measurements of the dependent variable `sigma`.

You supply a function that takes the independent variable as it first variable and any additional parameters as additional arguments. This function is one that you expect to fit your data. In our case we will use the black body spectrum `bb_spectrum_freq`

**The output of the fitting process is the parameters of this function that best fit your data, and some error bound on those parameters**


We can also seed the fitting process with a good guess of our parameters. In our case we can use the temperature $T_{peak}$ that we computed above that we computed above.

In [ ]:
#Run this cell

parameters, covariance = curve_fit(
    f=bb_spectrum_freq,  # this is a function that computes the theoretical irradiance at any frequency for a given temp
    xdata=freqs,   # this is the list of frequencies at which irradiance was measures
    ydata=bs,      # this is the measure irradiance
    sigma=sigmas,  # this is the standard error of each data point
    p0=[T_peak],   # this is the initial guess we give the fitting algorithm
)
T_fit = parameters[0]
sigma_T_fit = np.sqrt(covariance[0][0])
print('T_fit = {:0.4f}K\nsigma_T = {:0.4}K'.format(T_fit, sigma_T_fit))

**Question 4**

What are the values of T and your uncertainty (sigma)? Does your initial estimate fall within this range?

Now plot the theoretical curve at `T=T_fit` against the data.

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(111)
theory_f = np.linspace(5e10, 8e11, 1000)
ax1.plot(
    theory_f,
    bb_spectrum_freq(theory_f, T_fit),
    '-k', label='theory, T={:0.3f}'.format(T_fit)
)
ax1.errorbar(freqs, bs, 2*3*sigmas, fmt='.r', label="COBE FIRAS") # plot 95% confidence error bars (+/- 3 sigma)
plt.legend()
plt.ylabel('B ($W m^{-2} Hz^{-1}str^{-1}$)')
plt.xlabel('$f (Hz)$')
plt.legend()
plt.show()

**Question 5**

How does this theoretical curve compare to the the one you initially found? Zoom in to a few points - does the theoretical curve fall within the bounds of the error bars? This experiment was very precise, you may have to zoom in far!

Answer:

**Question 6**

Why do you think the temperature we determined with this data is a better match for the data? How could you quantify how closely a theoretical curve matches experimentally observed data?

Answer: